<a href="https://colab.research.google.com/github/NaijingGuo/SeniorDesign_FallPreventionML/blob/main/Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import tensorflow as tf
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/"+"data_df" + ".csv", skiprows=[0])
dataset = df.values
X = dataset[:,1:61].astype(float)
Y = dataset[:,61]
X = (X+90)/180
# randomize
num_inputs = len(X)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)
X = X[randomize]
Y = Y[randomize]

split = int(0.9* num_inputs)
X_train, X_test = np.split(X, [split])
Y_train, Y_test = np.split(Y, [split])

Mounted at /content/drive


MLP

In [4]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100, activation='relu', input_dim = 60)) # relu is used for performance
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid')) # we either have fall or not
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=100)

Epoch 1/100
18/18 [==============================] - 1s 4ms/step - loss: 0.6163 - accuracy: 0.7983
Epoch 2/100
18/18 [==============================] - 0s 7ms/step - loss: 0.5934 - accuracy: 0.8000
Epoch 3/100
18/18 [==============================] - 0s 6ms/step - loss: 0.5764 - accuracy: 0.8000
Epoch 4/100
18/18 [==============================] - 0s 5ms/step - loss: 0.5654 - accuracy: 0.8000
Epoch 5/100
18/18 [==============================] - 0s 3ms/step - loss: 0.5509 - accuracy: 0.8000
Epoch 6/100
18/18 [==============================] - 0s 6ms/step - loss: 0.5262 - accuracy: 0.8000
Epoch 7/100
18/18 [==============================] - 0s 6ms/step - loss: 0.5159 - accuracy: 0.8035
Epoch 8/100
18/18 [==============================] - 0s 4ms/step - loss: 0.4989 - accuracy: 0.8070
Epoch 9/100
18/18 [==============================] - 0s 3ms/step - loss: 0.4922 - accuracy: 0.8330
Epoch 10/100
18/18 [==============================] - 0s 5ms/step - loss: 0.4591 - accuracy: 0.8157
Epoch 11/

Test

In [5]:
predictions = model.predict(X_test)
predictions = predictions >= 0.5
from sklearn.metrics import classification_report
print(classification_report(Y_test, predictions))

from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
specificity = tn / (tn+fp)
print("specificity: "+ str(specificity))

              precision    recall  f1-score   support

         0.0       0.98      0.91      0.95        47
         1.0       0.80      0.94      0.86        17

    accuracy                           0.92        64
   macro avg       0.89      0.93      0.90        64
weighted avg       0.93      0.92      0.92        64

specificity: 0.9148936170212766


In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)
import os
basic_model_size = os.path.getsize("model.tflite")
print("Model is %d bytes" % basic_model_size)
!apt-get -qq install xxd
!echo "const unsigned char model[] = {" > /content/model.h
!cat model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

INFO:tensorflow:Assets written to: /tmp/tmph93k63d_/assets


INFO:tensorflow:Assets written to: /tmp/tmph93k63d_/assets


Model is 46640 bytes
Header file, model.h, is 287,648 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.


# New Section